import modules, define datatypes & import data

In [1]:
import numpy as np
import pandas as pd

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 
              'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 
              'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 
              'id':str, 'sqft_lot':int, 'view':int}

In [3]:
data = pd.read_csv('/Users/Andy/Documents/Coursera/kc_house_data_small.csv')
train = pd.read_csv('/Users/Andy/Documents/Coursera/kc_house_data_small_train.csv')
test = pd.read_csv('/Users/Andy/Documents/Coursera/kc_house_data_small_test.csv')
validation = pd.read_csv('/Users/Andy/Documents/Coursera/kc_house_data_validation.csv')

define functions

In [4]:
def get_numpy_data(df, features, output):
    df['constant'] =1
    features=['constant']+features
    features_df=df[features]
    feature_matrix=features_df.as_matrix()
    output_array=df['price']
    output_array=output_array.as_matrix()
    return(feature_matrix,output_array)

In [5]:
def normalize_features(feature_matrix):
    norms=np.linalg.norm(feature_matrix,axis=0)
    normalized=(feature_matrix)/norms
    return norms, normalized

In [6]:
#test normalize_features() function
normalized, norms = normalize_features(np.array([[3.,6.,9.,],[4.,8.,12.]]))
print(norms)
print(normalized)

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


create feature matrices for train, test and validation data

In [7]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']

In [8]:
features_train, output_train = get_numpy_data(train,feature_list,'price')
features_test, output_test = get_numpy_data(test,feature_list,'price')
features_val, output_val = get_numpy_data(validation,feature_list,'price')

In [9]:
norms, normalized_train = normalize_features(features_train)
features_test = features_test/norms
fearures_val = features_val/norms

Compute a single distance

In [10]:
print(normalized_train[0])
print(normalized_train[9])

[ 0.01345102  0.01163464  0.00602491  0.00703685  0.00181386  0.0085295   0.
  0.          0.0116321   0.01216718  0.00811747  0.          0.01333931
  0.          0.01343685 -0.01345623  0.00861561  0.00229178]
[ 0.01345102  0.01163464  0.00602491  0.0083488   0.00050756  0.01279425
  0.          0.          0.01938684  0.01390535  0.0096309   0.
  0.01302544  0.          0.01346821 -0.01346251  0.01195898  0.00156612]


What is the euclidean distance between <code>normalized_train[0]<code/> and <code>normalized_train[9]<code/> ?

In [11]:
#test 
a1 = np.array([3.,10.,5.])
a2 = np.array([4.,7.,15.])

print(a1-a2)
print((a1-a2)**2)
print((np.sum(a1-a2)**2))

[ -1.   3. -10.]
[   1.    9.  100.]
64.0


In [32]:
dist = np.sqrt(np.sum((features_test[0]-normalized_train[9])**2))

In [34]:
print('The Euclidean distance is',np.round(dist, decimals=5))

The Euclidean distance is 0.05972


In [35]:
#helper function for Euclidean distance
def euclid_dist(h1,h2):
    d = np.sqrt(np.sum((h1-h2)**2))
    return d

In [36]:
query_house = features_test[0]
first10 = normalized_train[0:10]

In [37]:
dist  = {}
for i in range(len(first10)):
    qh = query_house
    h_i = first10[i]
    d_i = euclid_dist(qh,h_i)
    dist[i] = d_i
    print(d_i)


print("Min distance is {0} for house number {1}".format(min(list(dist.values())),min(dist, key=dist.get)))

0.060274709163
0.0854688114764
0.0614994643528
0.0534027397929
0.0584448406017
0.0598792150981
0.0546314049678
0.0554310832361
0.0523836278402
0.059723593714
Min distance is 0.052383627840220305 for house number 8


Perform 1-nearest neighbor regression

In [16]:
diff = normalized_train - features_test[0]

In [17]:
#verify diff 
diff[-1].sum()

-0.093433998746546426

In [18]:
np.sum(diff**2,axis=1)

array([ 0.00363304,  0.00730492,  0.00378218, ...,  0.0032681 ,
        0.00325555,  0.00341846])

In [19]:
#verify sum
print(np.sum(diff**2,axis=1)[15])
print((np.sum(diff[15]**2)))

0.00330705902846
0.00330705902846


In [20]:
distances = np.sqrt(np.sum((normalized_train - features_test[0])**2,axis=1))

In [21]:
def compute_distances(feature_instances, features_query):
    distances = np.sqrt(np.sum((feature_instances-features_query)**2,axis=1))
    return distances

In [90]:
#quiz question #16
d = compute_distances(normalized_train, features_test[2:3])
print('The closest distanct is at location',np.where(d==min(d))[0][0])

The closest distanct is at location 382


In [56]:
#quiz question #17
print('The price of the nearest house is ${}'.format(np.round(output_train[382])))

The price of the nearest house is $249000


In [53]:
def k_nearest_neighbors(k, features_train, features_query):
    d = compute_distances(features_train, features_query)
    di = np.argsort(d)
    nn = di[0:k]
    return nn

In [55]:
#question 20
k_nearest_neighbors(4, normalized_train, features_test[2])

array([ 382, 1149, 4087, 3142])

In [192]:
def predict_output_of_query(k, features_train, output_train, features_query):
    i = k_nearest_neighbors(k, features_train, features_query)
    avgP = np.average(output_train[i])
    return avgP

In [197]:
predict_output_of_query(k=4, 
                        features_train=normalized_train, 
                        output_train=output_train, 
                        features_query=features_test[2:3])

413987.5

In [214]:
def predict_output(k, features_train, output_train, features_query):
    preds=[]
    for i in range(features_query.shape[0]):
        predict_i = predict_output_of_query(k, features_train, output_train, features_query[i])
        preds.append(predict_i)
    predictions=np.asarray(preds)
    return predictions

In [217]:
p = predict_output(k=10, features_train=normalized_train, 
               output_train=output_train, features_query=features_test[0:10])
np.argsort(p)[0]

6

In [233]:
allRSS = []
for i in range(1,16):
    p = predict_output(k=i,
                       features_train=normalized_train,
                       output_train=output_train,
                       features_query=features_val)
    resids = p-output_val
    rss = np.sum(resids**2)
    allRSS.append(rss)

In [234]:
allRSS_array = np.asarray(allRSS)
np.argsort(allRSS_array)

array([14, 13, 12, 11,  9, 10,  8,  7,  6,  5,  4,  3,  0,  2,  1])

In [231]:
min(allRSS)

211413021972473.56

In [1]:
min(allRSS)

NameError: name 'allRSS' is not defined